In [1]:
import sys, os
import copy

import torch
from torch import nn
from torch.nn.utils import spectral_norm
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable


In [2]:
def generator_hinge_loss(outputs):
    return -torch.mean(outputs)

def discriminator_real_hinge_loss(outputs):
    zeros = torch.zeros(len(outputs), 1)
    minval = torch.min(outputs - 1, zeros)
    return -torch.mean(minval)

def discriminator_fake_hinge_loss(outputs):
    zeros = torch.zeros(len(outputs), 1)
    minval = torch.min(-outputs - 1, zeros)
    return - torch.mean(minval)


In [3]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        # image size = 1 * 20 * 20 = 400
        # self.image_size = 14
        self.image_size = 20
        self.batch_size = 1
        # latent size
        latent_size = 16
        self.fc1 = nn.Linear(self.image_size * self.image_size, 64)
        self.fc2 = nn.Linear(64, latent_size)
        self.fc3 = nn.Linear(latent_size, 64)
        self.fc4 = nn.Linear(64, self.image_size * self.image_size)
        self.elu = nn.ReLU(.5)

        self.latent = None


    def encode(self, x):
        self.batch_size = x.size(0)
        x = x.view(self.batch_size, -1)
        # print(x.shape)
        x = self.fc1(x)
        # print(x.shape)
        x = self.elu(x)
        # print(x.shape)
        x = self.fc2(x)
        x = self.elu(x)
        self.latent = x
        return x

    def decode(self,x):
        x = self.fc3(x)
        # print(x.shape)
        x = self.elu(x)
        x = self.fc4(x)
        # print(x.shape)
        x = x.view(self.batch_size, 1, self.image_size, self.image_size)
        return x

    def forward(self, x):
        x = self.encode(x)
        # print(x.shape)
        x = self.decode(x)
        return x


In [4]:
class Aligner(nn.Module):
    def __init__(self):
        super(Aligner, self).__init__()
        # self.image_size = 14
        self.image_size = 20
        self.batch_size = 1
        # read layer
        self.fc1 = nn.Linear(self.image_size * self.image_size, self.image_size * self.image_size)

        # exp unit
        self.relu = nn.ReLU(.5)

        # out layer
        self.fc2 = nn.Linear(self.image_size * self.image_size, self.image_size * self.image_size)

    def forward(self, x):
        self.batch_size = x.size(0)
        x = x.view(self.batch_size, -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = x.view(self.batch_size, 1, self.image_size, self.image_size)
        return x

In [5]:
class Classifier(nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(16, 120)
        #self.fc1 = nn.Linear(16, 12)
        self.fc2 = nn.Linear(120, 12)
        self.relu = nn.ReLU(.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [6]:
def calc_vaf(x, y):
    x_dev = x - torch.mean(x,dim=0)
    y_dev = y - torch.mean(y,dim=0)
    return 1 - torch.sum(torch.square(x_dev - y_dev)) / torch.sum(torch.square(x_dev))



In [7]:
from functools import partial


def pairwise_distance(x, y):

    if not len(x.shape) == len(y.shape) == 2:
        raise ValueError('Both inputs should be matrices.')

    if x.shape[1] != y.shape[1]:
        raise ValueError('The number of features should be the same.')

    x = x.view(x.shape[0], x.shape[1], 1)
    y = torch.transpose(y, 0, 1)
    output = torch.sum((x - y) ** 2, 1)
    output = torch.transpose(output, 0, 1)

    return output

def gaussian_kernel_matrix(x, y, sigmas):

    sigmas = sigmas.view(sigmas.shape[0], 1)
    beta = 1. / (2. * sigmas)
    dist = pairwise_distance(x, y).contiguous()
    dist_ = dist.view(1, -1)
    s = torch.matmul(beta, dist_)

    return torch.sum(torch.exp(-s), 0).view_as(dist)

def maximum_mean_discrepancy(x, y, kernel= gaussian_kernel_matrix):

    cost = torch.mean(kernel(x, x))
    cost += torch.mean(kernel(y, y))
    cost -= 2 * torch.mean(kernel(x, y))

    return cost

def mmd_loss(source_features, target_features):

    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    
    gaussian_kernel = partial(
            gaussian_kernel_matrix, sigmas = Variable(torch.FloatTensor(sigmas))
        )
    loss_value = maximum_mean_discrepancy(source_features, target_features, kernel= gaussian_kernel)
    loss_value = loss_value

    return loss_value

In [8]:
# ------- Define Model ------- #

AE_model = AE()
classifier = Classifier()
aligner = Aligner()
discriminator = AE()



In [9]:
# ------- Optimizer ------- #


#AE_optimiser = optim.Adam(AE_model.parameters(), lr=0.001)

AE_optimiser = optim.SGD(AE_model.parameters(),lr= 0.001, momentum= 0.9)

classifier_criterion = nn.CrossEntropyLoss()

#classifier_optimiser = optim.Adam(classifier.parameters(), lr=0.001)
classifier_optimiser = optim.SGD(classifier.parameters(),lr= 0.001, momentum= 0.9)

#aligner_optimiser = optim.Adam(aligner.parameters(), lr=0.0002)
aligner_optimiser = optim.SGD(aligner.parameters(), lr=0.0002, momentum= 0.9)

#discriminator_optimiser = optim.Adam(discriminator.parameters(), lr=0.0002)
discriminator_optimiser = optim.SGD(discriminator.parameters(), lr=0.0002, momentum= 0.9)


In [11]:
AE_criterion = nn.MSELoss()
classifier_criterion = nn.CrossEntropyLoss()


In [12]:
from load_dataset_ADAN import load_dataset, get_data_by_ids, PID, get_key_from_val, DAY, DAY_NAME

p_rate=0.02

dataset_np = load_dataset(one_hot_label=False, cache=False)
source_id = DAY[0]
target_id = DAY[1]
    
source_dataset = get_data_by_ids(dataset_np,source_id)
source_test_size = 0.5

target_dataset = get_data_by_ids(dataset_np,target_id)
target_test_size = 0.5
target_p_size = p_rate
    


FileNotFoundError: [Errno 2] No such file or directory: '/Users/andy42i/Documents/Documentos/Python/datasets_ADAN/RFID_Activity_Location_PIDLabels.csv'

In [12]:

# prepare target data
target_train_images, target_test_images, target_train_labels, target_test_labels = train_test_split(target_dataset['image'],target_dataset['location_label'], test_size=target_test_size, stratify=target_dataset['location_label'])
target_p_train_images, target_p_test_images, target_p_train_labels, target_p_test_labels = train_test_split(target_train_images,target_train_labels, test_size=(1-target_p_size), stratify=target_train_labels)

# Prepare source data
train_images, test_images, train_labels, test_labels = train_test_split(source_dataset['image'],source_dataset['location_label'], test_size=source_test_size, stratify=source_dataset['location_label'])
train_images = np.concatenate([source_dataset['image'],target_p_train_images],axis=0)
train_labels = np.concatenate([source_dataset['location_label'],target_p_train_labels],axis=0)

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(train_images).float(),torch.from_numpy(train_labels.astype(np.int64)))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(test_images).float(),torch.from_numpy(test_labels.astype(np.int64)))

target_train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(target_p_train_images).float(),torch.from_numpy(target_p_train_labels.astype(np.int64)))
target_test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(target_test_images).float(),torch.from_numpy(target_test_labels.astype(np.int64)))


In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset, num_workers=2, batch_size=16, shuffle=True, drop_last=True)
target_train_loader = torch.utils.data.DataLoader(target_train_dataset, num_workers=2, batch_size=16, shuffle=True, drop_last=True)
    
fixed_ae_train_losses = []
total_train_losses = []
fixed_ae_test_losses = []
total_test_losses = []
cl_tr_losses = []
cl_tr_accuracy = []
cl_test_losses = []
cl_test_accuracy = []
overall_accuracy = []
NUM_EXP = 5

for ne in range(NUM_EXP):
    # Initialize adjust_loss
    adjust_loss = torch.tensor(1, dtype=torch.float)
    for epoch in range(300):  # loop over the dataset multiple times
        total_tr_losses = []
        cl_tr_loss = []
        ae_tr_loss = []
        size = 0
        correct = 0
        ae_running_loss = 0.0
        total_running_loss = 0.0

        # Training
        AE_model.train()
        classifier.train()
        aligner.train()
        #discriminator.train()

        for i, data in enumerate(train_loader, 0):
        #for i in range(1):
            # get the inputs; data is a list of [inputs, labels]
            _adjust_loss = adjust_loss
            inputs, labels = data

            # zero the parameter gradients
            AE_optimiser.zero_grad()
            classifier_optimiser.zero_grad()

            # forward + backward + optimize

            # calc auto encoder loss
            outputs = AE_model(inputs)
            ae_loss = AE_criterion(inputs, outputs)

            # calc classifier loss
            cl_ae_outputs = AE_model(inputs)
            cl_ae_loss = AE_criterion(inputs, outputs)
            ae_tr_loss.append(cl_ae_loss)

            cl_outputs = classifier(AE_model.latent.clone())
            cl_loss = classifier_criterion(cl_outputs, labels)
            cl_tr_loss.append(cl_loss)

            # calc total loss
            total_loss = _adjust_loss * cl_ae_loss + cl_loss
            total_tr_losses.append(total_loss)
            total_loss.backward()

            AE_optimiser.step()
            classifier_optimiser.step()

            ae_running_loss += ae_loss.item()
            total_running_loss += total_loss.item()

            _, predicted = cl_outputs.max(1)
            correct += (predicted == labels).sum().item()
            size += labels.size(0)

            ae_running_loss = 0.0
            total_running_loss = 0.0


    ###################
    model = copy.deepcopy(AE_model.state_dict())
    discriminator.load_state_dict(model)
    ones = torch.tensor(1, dtype=torch.float)
    minus_ones = ones * -1

    theta1 = 1
    theta2 = 0.5

    aligner_train_losses = []
    aligner_train_accuracy = []
    aligner_test_losses = []
    aligner_test_accuracy = []

    for epoch in range(300):  # loop over the dataset multiple times
        tr_aligner_losses = []
        tr_discriminator_losses = []
        tr_classifier_losses = []
        test_classifier_losses = []
        size = 0
        correct = 0
        AE_model.train()
        classifier.train()
        aligner.train()
        discriminator.train()

        target_train_loader = torch.utils.data.DataLoader(target_train_dataset, num_workers=2, batch_size=16, shuffle=True, drop_last=True)

        for i, data in enumerate(target_train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # Train aligner
            aligner_optimiser.zero_grad()
            discriminator_optimiser.zero_grad()
            aligned = aligner(inputs)
            aligned_img_tensor = aligned.detach()
            outputs_align = discriminator(aligned)

            # classified
            ae_out = AE_model(aligned)
            cl_outputs = classifier(AE_model.latent.clone())
            cl_loss = classifier_criterion(cl_outputs, labels)
            tr_classifier_losses.append(cl_loss)
            _, predicted = cl_outputs.max(1)
            correct += (predicted == labels).sum().item()
            size += labels.size(0)

            # loss
            output_aligned_r = outputs_align.view(outputs_align.shape[0],outputs_align.shape[2]**2)
            aligned_r = aligned.view(aligned.shape[0],aligned.shape[2]**2)

            mmdloss = cl_loss*theta1 + mmd_loss(output_aligned_r.float(),aligned_r.float()) * theta2 # + mmd_loss(predicted.float(), labels.float()) * theta2

            #loss_align = cl_loss + torch.mean((torch.abs(outputs_align - aligned)))

            loss_align = mmdloss

            tr_aligner_losses.append(loss_align)
            loss_align.backward(retain_graph=True)
            aligner_optimiser.step()

            # Train discriminator

            # zero the parameter gradients
            discriminator_optimiser.zero_grad()
            aligner_optimiser.zero_grad()

            # load source data
            orig_dataset_iter = iter(train_loader)
            originals = orig_dataset_iter.next()[0]
            outputs_orig = discriminator(originals)
            outputs_align = discriminator(aligned_img_tensor)

            originals_r = originals.view(originals.shape[0],originals.shape[2]**2)
            outputs_orig_r = outputs_orig.view(outputs_orig.shape[0],outputs_orig.shape[2]**2)

            aligned_img_tensor_r = aligned_img_tensor.view(aligned_img_tensor.shape[0],aligned_img_tensor.shape[2]**2)

            loss_orig = mmd_loss(originals_r.float(),outputs_orig_r.float()) * theta1 #torch.mean((torch.abs(outputs_orig - originals)))  

            loss_align_r = mmd_loss(output_aligned_r.float(),aligned_img_tensor_r.float()) * theta1  #torch.mean((torch.abs(outputs_align - aligned_img_tensor)))

            # loss
            loss_diff = loss_orig - loss_align_r
            tr_discriminator_losses.append(loss_diff)
            loss_diff.backward(retain_graph=True)
            discriminator_optimiser.step()

        # output metrics...
        accuracy = float(correct/ size)
        print(accuracy)
        avg_aligner_loss = float(sum(tr_aligner_losses) / len(tr_aligner_losses))
        avg_disc_loss = float(sum(tr_discriminator_losses) / len(tr_discriminator_losses))
        avg_cl_loss = float(sum(tr_classifier_losses) / len(tr_classifier_losses))

        aligner_train_losses.append(avg_aligner_loss)
        aligner_train_accuracy.append(accuracy * 100)
        #print(f'Train Epoch: [{epoch}] Align-loss: {loss_align} Accuracy: {accuracy} ({correct}/{size})')


    ####################
    target_loader_v = torch.utils.data.DataLoader(target_test_dataset, num_workers=2, batch_size=len(target_test_dataset))

    data_target = iter(target_loader_v).next()
    inputs_t, labels_t = data_target

    inputs = inputs_t

    AE_model.eval()
    classifier.eval()
    aligner.eval()
    discriminator.eval()

    # classified aligned images
    aligned = aligner(inputs)
    outputs = AE_model(aligned)
    cl_outputs = classifier(AE_model.latent.clone())
    _, predicted = cl_outputs.max(1)
    correct = (predicted == labels_t).sum().item()
    size = labels_t.size(0)
    accuracy = float(correct / size)
    print(accuracy)

     # classified raw(non-aligned) images
    raw_outputs = AE_model(inputs)
    raw_cl_outputs = classifier(AE_model.latent.clone())
    _, raw_predicted = raw_cl_outputs.max(1)
    raw_correct = (raw_predicted == labels_t).sum().item()
    raw_size = labels_t.size(0)
    raw_accuracy = float(raw_correct / raw_size)

    print(raw_accuracy)

    overall_accuracy.append(('Aligned Images', accuracy, 'Non-aligned images', raw_accuracy))

0.041666666666666664
0.125
0.22916666666666666
0.1875
0.125
0.25
0.2708333333333333
0.3125
0.3541666666666667
0.4166666666666667
0.3333333333333333
0.4791666666666667
0.3541666666666667
0.5416666666666666
0.6041666666666666
0.625
0.5833333333333334
0.4791666666666667
0.5833333333333334
0.5625
0.6666666666666666
0.625
0.5833333333333334
0.6458333333333334
0.7083333333333334
0.6041666666666666
0.625
0.5
0.6458333333333334
0.6041666666666666
0.7083333333333334
0.6041666666666666
0.6458333333333334
0.6458333333333334
0.6875
0.6875
0.6041666666666666
0.6875
0.7291666666666666
0.625
0.7083333333333334
0.6666666666666666
0.7708333333333334
0.6875
0.6875
0.6875
0.8125
0.7291666666666666
0.75
0.6458333333333334
0.75
0.7083333333333334
0.7291666666666666
0.6458333333333334
0.625
0.6458333333333334
0.7291666666666666
0.6458333333333334
0.6458333333333334
0.625
0.6041666666666666
0.6666666666666666
0.7291666666666666
0.6666666666666666
0.625
0.6458333333333334
0.625
0.625
0.7083333333333334
0.7291

0.9791666666666666
0.9375
0.8958333333333334
0.8958333333333334
0.9166666666666666
0.8958333333333334
0.9375
0.9583333333333334
0.9583333333333334
0.9375
0.9166666666666666
0.8958333333333334
0.8958333333333334
0.9166666666666666
0.8541666666666666
0.8958333333333334
0.8333333333333334
0.8333333333333334
0.7291666666666666
0.7083333333333334
0.7708333333333334
0.75
0.8125
0.875
0.8125
0.8125
0.8125
0.7708333333333334
0.7291666666666666
0.7708333333333334
0.8958333333333334
0.8541666666666666
0.8125
0.9166666666666666
0.8541666666666666
0.9166666666666666
0.9166666666666666
0.9166666666666666
0.9791666666666666
0.9583333333333334
0.9583333333333334
0.9375
0.9791666666666666
0.8541666666666666
0.9375
0.9166666666666666
0.9583333333333334
0.5571276972055182
0.5981605942695437
0.9166666666666666
0.9375
0.9583333333333334
0.9583333333333334
0.9375
0.8958333333333334
0.9375
0.9375
0.9166666666666666
0.9375
0.9791666666666666
0.9375
0.9375
0.9166666666666666
0.8958333333333334
0.8125
0.833333

0.875
0.8333333333333334
0.8958333333333334
0.6875
0.75
0.7708333333333334
0.9166666666666666
0.8541666666666666
0.875
0.8333333333333334
0.875
0.9166666666666666
0.8958333333333334
0.8541666666666666
0.9583333333333334
0.9375
0.875
0.9375
0.9375
0.9375
0.9583333333333334
0.9375
0.9166666666666666
0.9375
0.9166666666666666
0.9375
0.9375
0.9583333333333334
0.9583333333333334
0.9166666666666666
0.9583333333333334
0.9166666666666666
0.8958333333333334
0.9583333333333334
0.9583333333333334
0.9375
0.9583333333333334
0.8958333333333334
0.9791666666666666
0.9583333333333334
0.9375
0.9583333333333334
0.9583333333333334
0.9583333333333334
0.9583333333333334
0.9791666666666666
0.9583333333333334
1.0
0.9583333333333334
0.9791666666666666
0.9791666666666666
1.0
0.9583333333333334
0.9791666666666666
0.9791666666666666
0.9791666666666666
0.9375
0.9583333333333334
0.9375
0.9791666666666666
0.9375
0.9375
0.9791666666666666
1.0
0.9583333333333334
0.9791666666666666
0.9375
0.9583333333333334
0.833333333

In [14]:
overall_accuracy

[('Aligned Images',
  0.5624336752741422,
  'Non-aligned images',
  0.5903784931022286),
 ('Aligned Images',
  0.5571276972055182,
  'Non-aligned images',
  0.5981605942695437),
 ('Aligned Images',
  0.5758754863813229,
  'Non-aligned images',
  0.6229218252564556),
 ('Aligned Images',
  0.5175097276264592,
  'Non-aligned images',
  0.6027591085956845),
 ('Aligned Images',
  0.5737530951538734,
  'Non-aligned images',
  0.6108949416342413)]

In [15]:
import pandas as pd

pd.DataFrame(overall_accuracy).to_csv("ADAN_MMD_3days_1days.csv")